<a href="https://colab.research.google.com/github/kr7/IntelligensModszerekTantargy/blob/main/Kiertekeles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Adatok betöltése**

In [ ]:
import numpy as np
data_with_labels = np.loadtxt('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data', delimiter=',')

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.names

column_names = []
with open("spambase.names") as f:
  for i in range(33):
    f.readline()

  for i in range(57):
    line = f.readline()
    column_names.append(line.split(":")[0])

**1. feladat: keresztvalidációs protokollok** 

Osztályozza az adatokat döntési fával és értékelje ki a modellt 10-szeres keresztvalidációval.

(a) Az *sklearn.model_selection* csomagbeli *StratifiedKFold* használatával határozza meg, hogy mely példányok kerülnek a tanító és teszt halmazokba a keresztvalidáció egyes "fordulóiban". Írassa ki a tanító és teszt példányok indexeit (sorszámait) az egyes "fordulókban"! Egészítse ki az alábbi kódot!

(b) Ellenőrizze, hogy ugyanazon index (sorszám) nem fordul elő a tanító és teszt indexek között! 

(c) Ön szerint miért volt szükség arra, hogy a split(...) metódus bemenetként megkapja az adatokat és azok osztálycímkéit, holott mindössze indexeket (sorszámokat) adott vissza, adatpéldányokat nem. Elég lenne a példányok számának ismerete a split(...) metódus által végzett funkcionalitás megvalósításához?

(d) Többször egymás után futtatva a kódot, ugyanazok a példányok kerülnek-e tanító és teszthalmazba a keresztvalidáció egyes fordulóiban? Állítsa a *random_state*-t, hogy többszöri futtatás során ugyanazon felosztásokat kapjuk! Miért előnyös a *random_state* állítása?

(e) A StratifiedKFold-tól kapott indexeket használva ossza fel az adatokat tanító és tesztadatokra és tanítson egy-egy döntési fát a keresztvalidáció minden fordulójában. A fa legnagyobb mélysége legfeljebb 3 legyen! Emlékeztetőül, a döntési fa tanításához előző órán ezt a kódot használtuk:

    from sklearn.tree import DecisionTreeClassifier
    
    model = DecisionTreeClassifier(max_depth=4)
    model.fit(data_train, labels_train)

Az egyes modelleket egy *models* elnevezésű listába tárolja el.

(f) Értékelje ki a modelleket az egyes fordulókban, és tárolja el a modellek pontosságát (helyesen osztályozott példányok arányát) egy *accuracies* elnevezésű listába! Emlékeztetőül, a tanítotott modell predikcióit a predict(...) függvénnyel tudjuk kiszámolni:

    pred = model.predict(data_test)

(g) Számolja ki a modell átlagos pontosságát, a pontosság szórását, minimumát és maximumát!

(h) A megadott kód segítségével jelenítse meg 10-szeres keresztvalidáció egyes fordulóiban kapott döntési fákat és hasonlítsa össze ezeket! Lát-e különbségetek? Mi lehet ennek az oka?

(i) Tanítson egy modellt az **összes** rendelkezésre álló adat felhasználásával *best_model* néven! Mit tud elmondani ennek a modellnek a pontosságáról?


In [ ]:
# (a)-(g) részfeladatokhoz kapcsolódó kiegészítendő kód

...IDE JÖN A KÓD...

skf = StratifiedKFold(n_splits = ...IDE JÖN A KÓD...)

data = ...IDE JÖN A KÓD...
labels = ...IDE JÖN A KÓD...

for train_index, test_index in skf.split(data, labels):
  ...IDE JÖN A KÓD...

In [ ]:
# (h) részfeladathoz kapcsolódó kód

from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydot
from IPython.display import Image
from google.colab import widgets

tb = widgets.TabBar([str(i) for i in range(10)], location='top')
for i in range(10):
  with tb.output_to(i):
    dot_data = StringIO()
    export_graphviz(models[i], out_file=dot_data, feature_names=column_names, \
                        class_names=['non spam', 'spam'], filled=True, rounded=True, \
                        special_characters=True)
    graph = pydot.graph_from_dot_data(dot_data.getvalue())
    display(Image(graph[0].create_png()))

**2. feladat: kiértékelési metrikák: Precision, Recall, F-measure**

Tanítson egy logisztikus regressziós modellet az előbbi spam-felismerési feladatra a megadott kód segítségével! A *predict_proba* metódust használva a logisztikus regresszió egy 0 és 1 közötti valószínűséget prediktál minden osztályra, amely azt adja meg, hogy mekkora eséllyel tartozik egy-egy példány az egyes osztályokba. Feltételezve, hogy 0.5 feletti valószínűség esetén a modell az adott e-mail-t spam-ként osztályozta, és azt, hogy a "spam" osztályt tekintjük a pozitív osztálynak,

(a) számolja ki az osztályozó precision-ét (azt, hogy a pozitívnak prediktált példányok mekkora része pozitív ténylegesen),

(b) recall-át (azt, hogy a ténylegesen pozitív példányok mekkora részét prediktálta a modell pozitívnak), és 

(c) f-measure-ét (precision és recall harmónikus közepét)!

(d) Írjon egy függvényt, amely tetszőleges küszöbszám mellett kiszámolja a precision-t és recall-t. 

(e) Az előbbi függvényt meghívva mentse le a precision és recall értékeket 0.1, 0.2, 0.3, 0.4, ... 0.9 küszöbszámok mellett egy *precisions* ill. *recalls* elnevezésű listába. A megadott kód segítségével ábrázolja a kapott precision és recall értékeket!

(f) A precision-recall görbe alatti terület használható a modell pontosságának mérésére. Ugyanakkor ez a mérék túlságosan optimista lehet(*), egy jobb eljárás a modell pontosságának mérésére az összes lehetséges recall szint melletti precision átlaga, amely egyben közelítése a precision-recall görbe alatti területnek. Számolja ki a modell pontosságát az sklearn.metrics csomagbeli average_precision_score függvényt használva! Ha szükséges, tanulmányozza az alábbi dokumentációt:

(*) https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html


In [ ]:
from sklearn.linear_model import LogisticRegression

test_indices = np.array(range(0,len(data),5))
train_indices = [i for i in range(len(data)) if i not in test_indices]

data_train   = data[train_indices]
data_test    = data[test_indices]
labels_train = labels[train_indices]
labels_test  = labels[test_indices] 

model = LogisticRegression()
model.fit(data_train, labels_train)

pred = model.predict_proba(data_test)

In [ ]:
...IDE JÖN A KÓD...

In [ ]:
# (e) részfeladathoz kapcsolódó kód precision és recall ábrázolására
plt.xlabel('Precision')
plt.ylabel('Recall')
plt.plot(precisions, recalls, linestyle='--', marker='o');

In [ ]:
...IDE JÖN A KÓD...

**3. feladat: precision, recall, f-measure 10-szeres keresztvalidáció mellett**

(a) Az előbbi kódokból kiindulva tanítson logisztikus regressziót spam felismerésre 10-szeres keresztvalidáció mellett. A keresztvalidáció minden fordulójában számolja ki a precision, recall és f-measure értékét 0.5-ös küszöbszám mellett, és mentse el ezeket *prec_lr*, *recall_lr* és *f_lr* listákba.

(b) Számolja ki a 10 fordulóban kapott precision, recall és f-measure átlagát! Számolja ki az átlagos precision és átlagos recall harmónikus közepét! Hasonlítsa össze az f-measure-k átlagával!

(c) Hogyan tudnánk precision-t, recall-t és f-measure-t számolni többosztályos osztályozási feladatoknál, ahol nem csak két osztály van?

In [ ]:
...IDE JÖN A KÓD...

In [ ]:
...IDE JÖN A KÓD...

In [ ]:
...IDE JÖN A KÓD...

**4. feladat: szignifikancivizsgálat t-próbával**

(a) Módosítsa a 3-dik feladatban elkészített kódot úgy, hogy fordulónként lementi a helyesen osztályozott példányok arányát az *accuracies_lr* elnevezésű tömbbe. Számolja ki az átlagos accuracy-t.

(b) A scipy.stats csomagbeli ttest_rel függvényt használva döntse el, hogy van-e szignifikáns különbség a két osztályozó teljesítménye között!

In [ ]:
...IDE JÖN A KÓD...

In [ ]:
...IDE JÖN A KÓD...